In [25]:
from mmpose.apis import init_pose_model, inference_bottom_up_pose_model, vis_pose_result

DATAPATH = "/media/joris/uca-msc-dsai/research_project/data/"
config_file = DATAPATH + "videopose3d_h36m_27frames_fullconv_supervised.py"
checkpoint_file = DATAPATH + "videopose_h36m_27frames_fullconv_supervised-fe8fbba9_20210527.pth"
pose_model = init_pose_model(
  config_file, checkpoint_file, device="cuda:0"
)  # or device='cuda:0'

image_name = DATAPATH + "group_photo.JPG"
# test a single image
pose_results, _ = inference_bottom_up_pose_model(pose_model, image_name)

# show the results
vis_pose_result(pose_model, image_name, pose_results, out_file="demo/vis_persons.jpg")


FileNotFoundError: file "/media/joris/uca-msc-dsai/research_project/data/videopose3d_h36m_27frames_fullconv_supervised.py" does not exist

In [26]:
# from spacepy import pycdf
import cdflib
import numpy as np
import time
import cv2
import plotly.express as px
from pathlib import Path

filepath = DATAPATH / Path(
  "s1/Poses_RawAngles_S1/S1/MyPoseFeatures/RawAngles/Directions 1.cdf"
)
filepath = DATAPATH / Path(
  "s1/Poses_D3_Positions_S1/S1/MyPoseFeatures/D3_Positions/Directions 1.cdf"
)
filepath = DATAPATH / Path(
  "s1/Poses_D3_Positions_mono_S1/S1/MyPoseFeatures/D3_Positions_mono/Directions 1.54138969.cdf"
)
filepath = DATAPATH / Path(
  "s1/Poses_D2_Positions_S1/S1/MyPoseFeatures/D2_Positions/Directions 1.54138969.cdf"
)
cdf = cdflib.CDF(filepath)

pose = cdflib.cdf_to_xarray(filepath).data_vars["Pose"].values
print(f"{pose.shape = }")  # dim 0 = number of frames - 1


vidpath = "/media/joris/uca-msc-dsai/research_project/data//s1/S1/Videos/Directions 1.54138969.mp4"
cap = cv2.VideoCapture(vidpath)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"{length = }")
print(f"{cap.get(cv2.CAP_PROP_FRAME_HEIGHT) = }")
print(f"{cap.get(cv2.CAP_PROP_FRAME_WIDTH) = }")


success = True
count = 0
while success & (count < 300):
  coord = pose[count].reshape(32, -1).astype(int)

  success, frame_bgr = cap.read()
  if count % 100 == 0:
    frame = frame_bgr[:, :, ::-1]

    side = 10
    for w in range(side):
      for h in range(side):
        frame[coord[:, 1] - w, coord[:, 0] - h] = [255, 0, 0]
        frame[coord[:, 1] + w, coord[:, 0] + h] = [255, 0, 0]
    px.imshow(frame).show()
  count += 1


pose.shape = (1383, 64)
length = 1384
cap.get(cv2.CAP_PROP_FRAME_HEIGHT) = 1002.0
cap.get(cv2.CAP_PROP_FRAME_WIDTH) = 1000.0


In [16]:
import sys
mmpose_folder = "/media/joris/uca-msc-dsai/research_project/mmpose/"
if mmpose_folder not in sys.path:
  sys.path.insert(0, mmpose_folder)
from pathlib import Path
from typing import Optional
import demo.body3d_two_stage_video_demo as demo
from dataclasses import dataclass
%reload_ext autoreload
%autoreload 2




In [23]:
@dataclass
class Args:
  det_config: str = ""
  det_checkpoint: str = ""
  pose_detector_config: Optional[str] = "https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth"
  pose_detector_checkpoint: Optional[str] = f"{mmpose_folder}configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py"
  pose_lifter_config: str = "https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth"
  pose_lifter_checkpoint: str = f"{mmpose_folder}configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_h36m_243frames_fullconv_supervised_cpn_ft.py"
  video_path: str = "/media/joris/uca-msc-dsai/research_project/data/extracted/S1/Videos/Directions 1.54138969.mp4"
  rebase_keypoint_height = True
  norm_pose_2d = True
  num_instances: int = -1
  show = False
  out_video_root: str = "vis_results"
  device = "cuda:0"
  det_cat_id: int = 1
  bbox_thr: float = 0.9
  kpt_thr: float = 0.3
  use_oks_tracking = True
  tracking_thr: float = 0.3
  radius: int = 8
  thickness: int = 2
  smooth = True
  smooth_filter_cfg: str = "configs/_base_/filters/one_euro.py"
  use_multi_frames = False
  online = False


det_config = f"{mmpose_folder}demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py"
det_checkpoint = (
  "https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth",
)

args = Args(det_config=det_config, det_checkpoint=det_checkpoint)
demo.main(args=args)


Stage 1: 2D pose detection.
Initializing model...
/media/joris/uca-msc-dsai/research_project/mmpose/demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py


TypeError: expected string or bytes-like object